In [1]:
import os

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Import Libraries
import numpy as np
import json

from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import img_to_array, load_img
from keras.applications.imagenet_utils import preprocess_input

In [4]:
#Upload VGG16
vgg19 = VGG19(weights='imagenet')

574726144/574710816 [==============================] - 4s 0us/step


In [5]:
vgg19.save('/content/drive/MyDrive/Car Damage Toolkit New/static/vgg19.h5')

In [7]:
import tensorflow as tf
vgg19 = tf.keras.models.load_model('/content/drive/MyDrive/Car Damage Toolkit New/static/vgg19.h5')

In [8]:
def prepare_image(img_path):
    img = load_img(img_path, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [9]:
y = prepare_image('/content/drive/MyDrive/Car Damage Toolkit New/DataSets/car_damage_dataset/01-whole/0001.jpg')

In [10]:
preds = vgg19.predict(y)

In [11]:
preds.shape

(1, 1000)

In [12]:
preds

array([[9.60437418e-09, 2.02764205e-08, 6.06965358e-08, 5.22825836e-08,
        9.95387524e-08, 9.13023541e-08, 4.30368381e-08, 1.65854331e-07,
        7.59401644e-08, 4.57371101e-07, 1.45589185e-08, 8.42812842e-09,
        6.68628841e-09, 1.58956386e-08, 1.67410397e-09, 1.56015432e-08,
        1.99523971e-08, 1.94104643e-09, 1.75130719e-08, 3.52410501e-09,
        6.39085496e-09, 9.75606707e-09, 2.49870400e-08, 6.00761965e-08,
        3.91801677e-07, 5.26384213e-07, 3.61024121e-07, 2.81145880e-08,
        1.62951412e-08, 1.03375140e-08, 1.29209923e-08, 8.97194874e-10,
        1.22715589e-08, 5.66497079e-07, 7.93103709e-07, 8.90741987e-08,
        3.96563792e-07, 3.00405425e-08, 6.81547476e-08, 1.05838666e-07,
        1.68581611e-08, 1.44626227e-07, 2.99494651e-08, 2.04485140e-08,
        8.32810798e-08, 6.87590727e-07, 1.30309301e-08, 4.12052259e-08,
        1.18734670e-06, 1.59563296e-08, 4.18948467e-08, 4.47414408e-07,
        4.38126548e-08, 8.44797725e-08, 2.56060332e-07, 5.614858

In [13]:
CLASS_INDEX = None

In [17]:
CLASS_INDEX_PATH = '/content/drive/MyDrive/Car Damage Toolkit New/static/imagenet_class_index.json'

In [18]:
def get_predictions(preds,top=5):
    global CLASS_INDEX
    
    #Load the json file
    CLASS_INDEX = json.load(open(CLASS_INDEX_PATH))
    
    #get the results
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
        result.sort(key=lambda x: x[2], reverse=True)
        results.append(result)
    return results

In [19]:
print (get_predictions(preds, top=5))

[[('n04461696', 'tow_truck', 0.66571236), ('n03594945', 'jeep', 0.11169245), ('n04252225', 'snowplow', 0.066141896), ('n03345487', 'fire_engine', 0.04814624), ('n03478589', 'half_track', 0.030427404)]]


In [20]:
#First Check - category list creation

In [21]:
from collections import Counter, defaultdict
import os
import pickle as pk

In [22]:
def get_car_categories():
    d = defaultdict(float)
    img_list = os.listdir('/content/drive/MyDrive/Car Damage Toolkit New/DataSets/car_damage_dataset/01-whole')
    for i, img_path in enumerate(img_list):
        img = prepare_image('/content/drive/MyDrive/Car Damage Toolkit New/DataSets/car_damage_dataset/01-whole/'+img_path)
        out = vgg19.predict(img)
        top = get_predictions(out, top=5)
        for j in top[0]:
            d[j[0:2]] += j[2]
        if i % 100 == 0:
            print (i, '/', len(img_list), 'complete')
    return Counter(d)

In [23]:
cat_counter=get_car_categories()

0 / 920 complete
100 / 920 complete
200 / 920 complete
300 / 920 complete
400 / 920 complete
500 / 920 complete
600 / 920 complete
700 / 920 complete
800 / 920 complete
900 / 920 complete


In [24]:
cat_counter.most_common()[0:8]

[(('n03770679', 'minivan'), 136.78895090240985),
 (('n04285008', 'sports_car'), 119.89040146674961),
 (('n03100240', 'convertible'), 74.37932709977031),
 (('n03930630', 'pickup'), 62.5587771985447),
 (('n02814533', 'beach_wagon'), 58.054536994954105),
 (('n04037443', 'racer'), 52.93018546697567),
 (('n03670208', 'limousine'), 42.237966875662096),
 (('n03594945', 'jeep'), 38.84352175355889)]

In [25]:
with open('/content/drive/MyDrive/Car Damage Toolkit New/static/cat_counter.pk', 'wb') as f:
   pk.dump(cat_counter,f,-1)

In [26]:
# This piece of code will be used to load already saved cat_counter
with open('/content/drive/MyDrive/Car Damage Toolkit New/static/cat_counter.pk', 'rb') as f:
    cat_counter = pk.load(f)

In [27]:
cat_list  = [k for k, v in cat_counter.most_common()[:27]]

In [28]:
cat_list

[('n03770679', 'minivan'),
 ('n04285008', 'sports_car'),
 ('n03100240', 'convertible'),
 ('n03930630', 'pickup'),
 ('n02814533', 'beach_wagon'),
 ('n04037443', 'racer'),
 ('n03670208', 'limousine'),
 ('n03594945', 'jeep'),
 ('n03769881', 'minibus'),
 ('n02974003', 'car_wheel'),
 ('n02930766', 'cab'),
 ('n04461696', 'tow_truck'),
 ('n03459775', 'grille'),
 ('n03796401', 'moving_van'),
 ('n04252225', 'snowplow'),
 ('n03977966', 'police_van'),
 ('n02701002', 'ambulance'),
 ('n04065272', 'recreational_vehicle'),
 ('n04467665', 'trailer_truck'),
 ('n03891332', 'parking_meter'),
 ('n03345487', 'fire_engine'),
 ('n03777568', 'Model_T'),
 ('n03791053', 'motor_scooter'),
 ('n02965783', 'car_mirror'),
 ('n03445924', 'golfcart'),
 ('n04252077', 'snowmobile'),
 ('n02704792', 'amphibian')]

In [29]:
def car_categories_gate(image_path, cat_list):
    img = prepare_image(image_path)
    out = vgg19.predict(img)
    top = get_predictions(out, top=5)
    print ("Validating that this is a picture of your car...")
    for j in top[0]:
        if j[0:2] in cat_list:
            print (j[0:2])
            return "Validation complete - proceed to damage evaluation"
    return "Are you sure this is a picture of your car? Please take another picture (try a different angle or lighting) and try again."

In [30]:
car_categories_gate('/content/drive/MyDrive/Car Damage Toolkit New/DataSets/car_damage_dataset/data2a/validation/00-front/0004.jpeg', cat_list)

Validating that this is a picture of your car...
('n03770679', 'minivan')


'Validation complete - proceed to damage evaluation'